# Assignment 1 – Glacier Mass Balance Model

Let's go!

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import rasterio as rio
from rasterio.plot import show

# Nr 1 – calculate the mass balance 1980-2009

In [38]:
dem_2009 = rio.open('data_raw/dem_fiescher_2009.asc')
dem_1980 = rio.open('data_raw/dem_fiescher_1980.asc')
mask_2009 = rio.open('data_raw/glacier_mask_fiescher_2009.asc')
mask_1980 = rio.open('data_raw/glacier_mask_fiescher_1980.asc')






AttributeError: module 'rasterio' has no attribute 'bounds'

**Question 1 [6 pts]: How large was the specific mass change rate experienced by the glacier
during the period?**

*Express your result in units of ”m water equivalent per year” (m w.e. a−1), and make sure to
explain what assumptions you did for computing this number.
Hint: For converting from kg to mw.e. you can consider the average glacier surface for the two
given years.*





# Nr 2 - choose and read in climate data

Station: 
 - Grimsel Hospiz (1980 MSL)
 - how high is Fieschergletscher?
 
Norm value charts for Precipitation:
 - Precip 61-90, precip 91-20 and sunshine 91-20
 - Yellow left Jungfraujoch, yellow right Grimsel, orange ~ location of Fieschergletscher
 - Glacier in the area with highest precipitation of switzerland, not entirely represented by either of the stations
 - Grimsel in data_raw folder
 - URL for data: https://www.meteoswiss.admin.ch/services-and-publications/applications/ext/climate-tables-homogenized.html 
 - URL if you want to look at more norm value maps: https://www.meteoswiss.admin.ch/services-and-publications/applications/ext/climate-norm-maps-public.html
 
<div>
<img src="maps/precip_61-90.png" width="700" align="left"/>
<img src="maps/precip_91-20.png" width="400" align="left"/>
<img src="maps/sunshine_91-20.png" width="400" align="left"/>
</div>
 
 

**Question 2 [5 pts]: Which data did you chose and why?**
*Make sure to state the url from which you downloaded the data, and to discuss the choice of
the station that you selected.*

We used homogeneous monthly temperature and precipitation data from MeteoSwiss (which can be found here: https://www.meteoswiss.admin.ch/services-and-publications/applications/ext/climate-tables-homogenized.html), as MeteoSwiss provides the official long-time station records for Switzerland. 

For Fieschergletscher, we considered the three closest stations: Jungfraujoch at 3571 MSL, Grimsel Hospiz (GRH) at 1980 MSL and Meiringen at 589 MSL. All stations go back further than 1935. However, only Grimsel Hospiz and Meiringen combine temperature and precipitation data, while Jungfraujoch provides only temperature. Therefore, the record would have to be combined with precipitation data from another station – which would lead to new uncertainties, taking into account elevation differences and how much precipitation varies even at small spatial scales.

Of the two remaining stations, we chose Grimsel Hospiz. The station is substantially closer to the glacier than Meiringen and unlike Meiringen at a similar elevation level as the Fieschergletscher. Moreover, it is the only station in the area with a similar yearly amount of precipitation as the glacier.

In [21]:
GRH = pd.read_csv("data_raw/climate_GRH_data.txt", sep='\s+')

GRH

,Year,Month,Temperature,Precipitation
0,1932,1,-2.8,216.8
1,1932,2,-9.0,24.6
2,1932,3,-5.2,169.0
3,1932,4,-3.2,137.0
4,1932,5,2.0,164.7
...,...,...,...,...
1096,2023,5,5.0,128.4
1097,2023,6,10.7,72.1
1098,2023,7,11.6,102.1
1099,2023,8,11.3,196.7


# Nr 3 – Calibrate a TI model

Use the meteorological time series retrieved in Question 2 to calibrate the Degree Day Factor
(DDF) of a simple temperature index model. For this task, you can make the following assumptions:

- Precipitation can be modelled with a ”block rain model” (see lecture notes) and the time series retrieved in Question 2.
- Snow and rain can be distinguished through a threshold temperature of T0 = 2 ◦C.
- The temperature at any elevation can be computed from the data retrieved in Task 2 and by using a temperature lapse rate of dT/dz = 6.5 ◦Ckm−1.
- Melt occurs for temperatures above 0 ◦C.
- Every month has 365/12 ≈ 30.42 days.
- The provided DEMs both refer to October.
- The glacier surface can be described with the DEM of 2009 and does not change in time.

TI-Model Review: https://www.sciencedirect.com/science/article/abs/pii/S0022169403002579?via%3Dihub 

In [95]:
mb = -0.5 # m w.e./a, replace with actual value


# annual_ablation = DDF ((T1 + 0.0065*delta_h)>0 +  T2...+T12)
# area constant, not necessary for ablation (if constant elevation is assumed)

# for glacier elevation = mean elevation 
# more accurate: calculate percentage of glacier area above/below 0
# assumed difference between mean glacier elevation and station: Glacier is 100 m higher
# delta_T = Temperature difference that needs to be substracted from T for equal elevation glacier

delta_T = 0.0065*100
GRH_processed = GRH.copy()
GRH_processed["T_glacier"] = GRH_processed["Temperature"] - delta_T


# dataframe with only rows that are months between 1980 and 2009 above 0 °C
GRH_processed = GRH_processed[(GRH_processed.T_glacier >= 0)]
GRH_processed = GRH_processed[(GRH_processed.Year >= 1980)]
GRH_processed = GRH_processed[(GRH_processed.Year <= 2009)]

# calculate days per year that count into melting season
GRH_processed["n_months"] = 1
GRH_processed = GRH_processed.groupby("Year").sum()
GRH_processed["melt_days"] = GRH_processed["n_months"] * 30.42

GRH_processed


# total ablation per year = DDF * melt_days * T_glacier = ((DDF*30.42*T1) + (DDF*30.42*T2) + ...)

# Version 1: assume flexible mass balances 1980-2009
# --> 1 DDF for all years
# 30*mb = sum(accumulation1980-2009) - DDF * sum(melt_days*T_glacier 1980-2009) 

# Version 2: assume same mass balance for every year 1980-2009
# --> different DDF for all years
# calculation for every year:
# mb = annual_accumulation - DDF * melt_days * T_glacier

# check in Literature whether version 1 or 2 is preferred!


,Month,Temperature,Precipitation,T_glacier,n_months,melt_days
Year,,,,,,
1980,45,30.8,887.2,26.90,6,182.52
1981,49,34.0,1180.6,29.45,7,212.94
1982,45,40.2,774.3,36.30,6,182.52
1983,56,43.8,827.7,39.25,7,212.94
1984,51,32.8,675.6,28.90,6,182.52
1985,45,43.3,600.4,39.40,6,182.52
1986,45,41.1,635.5,37.20,6,182.52
1987,40,37.6,749.1,34.35,5,152.10
1988,49,45.3,826.9,40.75,7,212.94


In [ ]:
for year in GRH_processed["Year"]:
    if year in range(1980,2010):
        print(year)

**Question 3.1 [7 pts]: What was your procedure to calibrate the DDF?**
*Make sure to provide a step by step explanation and to mention any assumption.*

- concerning Hint 3: while TI models work robust at big scales (e.g. entire catchment), they miss local effects and varying melt rates that can be caused by different exposure to radiation/differences in debris cover of different parts of the glacier

**Question 3.2 [6 pts]: What value (and unit) do you obtain for the calibrated DDF, and how
does it compare to values reported in the literature?**
*Make sure to have ”days” somewhere in the units for the ”Degree Day Factor”, and compare
your result against at least two published DDF values (provide the corresponding references).
Discuss what might be at the origin of the differences that you find (if any).*

- Value from assignment sheet: DDF = 5 mmw.e. (◦Cd)−1.

**Question 3.3 [6 pts]: Reflect upon the various assumptions that were necessary for the calibration.**
*Which assumptions are likely to introduce a bias? In what direction do the individual
biases go, and which assumption are likely to introduce the largest biases?*

- if Nov-Apr main accumulation season and May-Sep main ablation season (what about October?): 
-> Summer is one day longer than model calculation, winter one day shorter. 


- **Hint 1:** What you want is to choose a DDF such that the model reproduces the total glacier mass change that you determined in Question 1.
- **Hint 2:** You can perform your computations at monthly resolution.
- **Hint 3:** Reflect whether you want to consider a distributed modelling approach (i.e. if you want to consider every grid-cell of the glacier separately) or not (e.g. if you only want to consider the mean glacier elevation, for example). Make sure to discuss this point in Question 3.1.

# Nr 4 – Mass balance reconstruction

Use the model calibrated in Task 3 and the same meteorological time series as above to reconstruct
the annual mass balance of the glacier for the period 1935-2022. Express the results in
mw.e. a−1, and refer the annual values to the hydrological year (1 Oct. – 30 Sep.). Plot your
results.

**Question 4 [7 pts]: What are the (i) cumulative and (ii) average glacier mass balance (in
mw.e. and mw.e. a−1, respectively) and what do you observe in the yearly time series?**
*Check whether you can discern trends, and whether there are periods or individual years that
stand out. For this task, you can neglect any changes in glacier geometry.*

# Nr 5 – Future projection
Use volume-area scaling to estimate the volume of the glacier for the year 2009. Then use the
mass balance time series that you computed in Task 4 for computing the mass change until 2022.
For the future, assume that the glacier mass balance rate (in mw.e. a−1) will remain the same
as it was on average over the past 30 years (period 1992-2022).

**Question 5.1 [4 pts]: What was the glacier volume in 2009?**
*Express you results in km3.*

**Question 5.2 [6 pts]: Based on your projection, when (year) will the glacier have lost 50% of
the volume it had in 2009?**
*Use ”Hint 1” below, and discuss whether the resulting ”projection” is likely to be an under- or
overestimate, and why.*

**Question 5.3 [4 pts]: Can you think of a strategy that avoids the simplification given in
”Hint 1”?**
*Explain how you could avoid the assumption of a constant area, and what additional information,
tools, or methods (if any) you would need for that.*

- Hint 1: As a simplification, you can assume that the glacier area remains constant at the 2009 level.
- Hint 2: If you did not solve Task 4, you can use the time series in the file sample massbalance time series.txt.
- Hint 3: Pay attention to the units for the glacier mass balance rate and the glacier volume.
- Hint 4: If you need to convert glacier volume to mass, assume a mean glacier density of 900 kgm−3.